In [12]:
import sys
# Force reinstall all packages to ensure they are properly loaded.
!pip install -U langchain langchain-core langchain-google-genai scikit-learn pandas
# Restart the kernel so that the newly installed packages are correctly picked up.
# This is a common practice after installing packages that might affect the current environment.
# However, since I cannot directly restart the kernel, I will only provide the reinstall command.
# If the issue persists, a manual kernel restart might be necessary.


In [13]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate


In [14]:
data = {
    "age": [22, 35, 45, 60],
    "bmi": [19, 28, 32, 30],
    "smoking": [0, 1, 1, 0],
    "risk": [0, 1, 1, 1]
}

df = pd.DataFrame(data)
df


,age,bmi,smoking,risk
0,22,19,0,0
1,35,28,1,1
2,45,32,1,1
3,60,30,0,1


In [15]:
X = df[["age", "bmi", "smoking"]]
y = df["risk"]

ml_model = LogisticRegression()
ml_model.fit(X, y)

print("ML model trained")


ML model trained


In [16]:
age = int(input("Enter age: "))
bmi = float(input("Enter BMI: "))
smoking = int(input("Smoking? (1 = Yes, 0 = No): "))


Enter age: 27
Enter BMI: 22
Smoking? (1 = Yes, 0 = No): 0


In [17]:
prediction = ml_model.predict([[age, bmi, smoking]])

risk = "High Risk" if prediction[0] == 1 else "Low Risk"
print("Predicted Risk:", risk)


Predicted Risk: Low Risk


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [18]:
from google.colab import userdata

api_key = userdata.get("GEMINI_API_KEY")

if api_key is None:
    raise ValueError("GEMINI_API_KEY not found in Colab Secrets")

print("API key loaded securely")


API key loaded securely


In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",  # Use a model from list_models()
    google_api_key=api_key
)

print("Gemini LLM initialized with models/gemini-2.5-flash")

Gemini LLM initialized with models/gemini-2.5-flash


In [20]:
import google.generativeai as genai
genai.configure(api_key=api_key)

print("Available models for generateContent:")
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

health_prompt = PromptTemplate(
    input_variables=["age", "bmi", "smoking", "risk"],
    template="""
You are an AI health assistant.

User information:
Age: {age}
BMI: {bmi}
Smoking habit: {smoking}
Predicted health risk: {risk}

Explain the risk in simple words.
Give exactly 3 lifestyle improvement tips.
Add a disclaimer that this is not medical advice.
"""
)


Available models for generateContent:
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-previ

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",  # Use a model from list_models()
    google_api_key=api_key
)

health_chain = health_prompt | llm

print(f"LLM is configured to use model: {llm.model}")

response = health_chain.invoke({
    "age": age,
    "bmi": bmi,
    "smoking": smoking,
    "risk": risk
})

print(response.content)

LLM is configured to use model: models/gemini-2.5-flash
Based on your current information (age 27, BMI 22.0, and no smoking habit), your **predicted health risk is low**.

This is great news! It means that, given these common indicators, you generally have a reduced likelihood of developing certain chronic health issues compared to someone with higher risk factors. It's a positive sign of your current health status.

Here are 3 lifestyle improvement tips to help you maintain your excellent health:

1.  **Prioritize a Balanced Diet:** Focus on incorporating a wide variety of colorful fruits and vegetables, whole grains, and lean proteins into your daily meals. Limiting processed foods, sugary drinks, and unhealthy fats can further support your long-term well-being.
2.  **Stay Consistently Active:** Aim for at least 150 minutes of moderate-intensity aerobic activity (like brisk walking, swimming, or cycling) or 75 minutes of vigorous activity each week, combined with strength training ex